## Research commodity options calendar spreads, especially in ES
### Try to optimize rolling portfolio protection

In [1]:
import zipfile
import glob
import pandas as pd
import numpy as np

from argparse import ArgumentParser
from argparse import RawDescriptionHelpFormatter
import sys
import os
if  not './' in sys.path:
    sys.path.append('./')
if  not '../' in sys.path:
    sys.path.append('../')

from barchartacs import build_db
from barchartacs import db_info
import plotly.graph_objs as go
from plotly.offline import  init_notebook_mode, iplot
init_notebook_mode(connected=True)
import plotly.tools as tls
from plotly.graph_objs.layout import Font,Margin
from IPython import display

import datetime
import io
from tqdm import tqdm,tqdm_notebook
from barchartacs import pg_pandas as pg
import mibian
import py_vollib
import importlib
from py_vollib import black
from py_vollib.black import implied_volatility
import ipdb
import traceback
import pandas_datareader.data as pdr

# importlib.reload(build_db)

/Users/bperlman1/Virtualenvs3/dashrisk4/lib/python3.6/site-packages/pandas_datareader/compat/__init__.py:7: FutureWarning:

pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.



## IF YOU WANT TO SEE WARNINGS, COMMENT THIS OUT

In [2]:
import warnings
warnings.filterwarnings("ignore")

### important global variables

In [3]:

DEBUG_IT=False
opttab = 'sec_schema.options_table'
futtab = 'sec_schema.underlying_table'
SYMBOL_TO_RESEARCH = 'ES'
STRIKE_DIVISORS = {}

# df_expiry_dates_additions = pd.read_csv('df_expiry_dates_additions.csv')
df_expiry_dates_additions = pd.read_csv('live_option_expirations.csv')


#### get all contracts in the options database

In [4]:
pga = db_info.get_db_info()
print(f"futtab max date: {pga.get_sql(f'select max(settle_date) from {futtab}')}")
print(f"opttab max date: {pga.get_sql(f'select max(settle_date) from {opttab}')}")


  sec_db
futtab max date:         max
0  20200311
opttab max date:         max
0  20200311


In [5]:
def plotly_plot(df_in,x_column,plot_title=None,
                y_left_label=None,y_right_label=None,
                bar_plot=False,figsize=(16,10),
                number_of_ticks_display=20,
                yaxis2_cols=None,
                x_value_labels=None):
    ya2c = [] if yaxis2_cols is None else yaxis2_cols
    ycols = [c for c in df_in.columns.values if c != x_column]
    # create tdvals, which will have x axis labels
    td = list(df_in[x_column]) 
    nt = len(df_in)-1 if number_of_ticks_display > len(df_in) else number_of_ticks_display
    spacing = len(td)//nt
    tdvals = td[::spacing]
    tdtext = tdvals
    if x_value_labels is not None:
        tdtext = [x_value_labels[i] for i in tdvals]
    
    # create data for graph
    data = []
    # iterate through all ycols to append to data that gets passed to go.Figure
    for ycol in ycols:
        if bar_plot:
            b = go.Bar(x=td,y=df_in[ycol],name=ycol,yaxis='y' if ycol not in ya2c else 'y2')
        else:
            b = go.Scatter(x=td,y=df_in[ycol],name=ycol,yaxis='y' if ycol not in ya2c else 'y2')
        data.append(b)

    # create a layout
    layout = go.Layout(
        title=plot_title,
        xaxis=dict(
            ticktext=tdtext,
            tickvals=tdvals,
            tickangle=45,
            type='category'),
        yaxis=dict(
            title='y main' if y_left_label is None else y_left_label
        ),
        yaxis2=dict(
            title='y alt' if y_right_label is None else y_right_label,
            overlaying='y',
            side='right'),
        margin=Margin(
            b=100
        )        
    )

    fig = go.Figure(data=data,layout=layout)
    fig.update_layout(
        title={
            'text': plot_title,
            'y':0.9,
            'x':0.5,
            'xanchor': 'center',
            'yanchor': 'top'})
    return fig

def plotly_shaded_rectangles(beg_end_date_tuple_list,fig):
    ld_shapes = []
    for beg_end_date_tuple in beg_end_date_tuple_list:
        ld_beg = beg_end_date_tuple[0]
        ld_end = beg_end_date_tuple[1]
        ld_shape = dict(
            type="rect",
            # x-reference is assigned to the x-values
            xref="x",
            # y-reference is assigned to the plot paper [0,1]
            yref="paper",
            x0=ld_beg[i],
            y0=0,
            x1=ld_end[i],
            y1=1,
            fillcolor="LightSalmon",
            opacity=0.5,
            layer="below",
            line_width=0,
        )
        ld_shapes.append(ld_shape)

    fig.update_layout(shapes=ld_shapes)
    return fig

In [6]:
def _next_monthyear_code(contract):
    code_val = contract[-3]
    code_num = DICT_MONTH_NUMS[code_val]
    y = int(contract[-2:])
    if code_num+1>12:
        next_code_num = 1
        next_y = y + 1
    else:
        next_code_num = code_num+1
        next_y = y
    next_code_val = MONTH_CODES[next_code_num-1]
    next_contract = contract[0:-3] + next_code_val + '%02d' %(next_y)
    return next_contract

def get_postgres_data(contract,strike_divisor=None):
    '''
    Get options and underlying data for ONLY ONE CONTRACT
    '''
    osql = f"select * from {opttab} where symbol='{contract}';"
    dfo = pga.get_sql(osql)
    if len(dfo)<10:
        e = f'''
        get_postgres_data ERROR: not enough option data for contract {contract} 
        '''
        raise ValueError(e)
    num_settle_days = len(dfo.settle_date.unique())
    u_contract = contract
    for i in range(12):
        usql = f"select * from {futtab} where symbol='{u_contract}';"
        dfu = pga.get_sql(usql)
        if len(dfu) < num_settle_days:
            u_contract = _next_monthyear_code(u_contract)
            print(f'trying contract {u_contract}')
        else:
            break

    if len(dfu)< num_settle_days:
        e = f'''
        get_postgres_data ERROR: not enough underlying days found for options contract {contract} 
        where len(underlying) = {len(dfu)} and num_settle_days = {num_settle_days}
        '''
        raise ValueError(e)
    # Merge options and futures data
    dfu = dfu.rename(columns={'symbol':'u_symbol'})
    df = dfo.merge(dfu,how='inner',on=['settle_date'])
    # Get options expiration dates
    df_expiry_dates = dfo[['symbol','settle_date']].groupby('symbol',as_index=False).max()
    df_additions = df_expiry_dates_additions[df_expiry_dates_additions.symbol==contract]
    df_additions = df_additions[['symbol','yyyymmdd_option']].rename(columns={'yyyymmdd_option':'settle_date'})
    additional_symbols = df_additions.symbol.values
    df_expiry_dates = df_expiry_dates[~df_expiry_dates.symbol.isin(additional_symbols)]
    df_expiry_dates = df_expiry_dates.append(df_additions).sort_values('symbol').copy()
    if strike_divisor is not None:
        df.strike = df.strike/strike_divisor
    return df,df_expiry_dates

### analyse option volumes by day, to see largest volumes

In [7]:
np.arange(17,20,1)

array([17, 18, 19])

In [8]:
#sql
commod = 'ES'
m = 'HUMZUHZM'
y = np.array([np.array([i,i,i,i,i,i+1,i,i+1]) for i in np.arange(17,20,1)])


myy_pairs = np.array(
                [np.array([f'{m[j]}{y[i][j]}' for j in range(len(m))]
            ) for i in range(len(y))])
myy_pairs = myy_pairs.reshape(-1).reshape(-1,2)
strike = 2500
for mp in myy_pairs:
    leg_front = f'{commod}{mp[0]}'
    leg_back = f'{commod}{mp[1]}'
    sql_select_options = f"""
    select o.settle_date,o.symbol,o.strike,o.pc,o.close o_close from {opttab} o 
    where symbol in ('{leg_front}','{leg_back}') 
    and o.strike={strike}
    join {futtab} f on o.symbol = f.symbol and o.settle_date = f.settle_date
    order by o.settle_date,o.symbol
    """

In [9]:
myy_pairs

array([['H17', 'U17'],
       ['M17', 'Z17'],
       ['U17', 'H18'],
       ['Z17', 'M18'],
       ['H18', 'U18'],
       ['M18', 'Z18'],
       ['U18', 'H19'],
       ['Z18', 'M19'],
       ['H19', 'U19'],
       ['M19', 'Z19'],
       ['U19', 'H20'],
       ['Z19', 'M20']], dtype='<U3')

In [10]:
mp = ['H19', 'U19']#myy_pairs[9]
strike = 2750.0
pc = 'P'
leg_front = f'{commod}{mp[0]}'
leg_back = f'{commod}{mp[1]}'
sql_select_options = f"""
select o.settle_date,o.symbol,o.strike,o.pc,o.close o_close,f.close f_close from {opttab} o 
join {futtab} f on o.symbol = f.symbol and o.settle_date = f.settle_date
where o.symbol in ('{leg_front}','{leg_back}') 
and o.strike={strike} and o.pc = '{pc}'
order by o.settle_date,o.symbol
"""
df_opt_pair = pga.get_sql(sql_select_options)
df_front = df_opt_pair[df_opt_pair.symbol==leg_front]
df_front = df_front.rename(columns={'symbol':'fr_symbol','o_close':'fr_opt_pr','f_close':'fr_fut_pr'})
df_back = df_opt_pair[df_opt_pair.symbol==leg_back]
df_back = df_back.rename(columns={'symbol':'bk_symbol','o_close':'bk_opt_pr','f_close':'bk_fut_pr'})
merge_cols = ['settle_date','strike','pc']
df_both = df_front.merge(df_back,how='inner',on=merge_cols)
df_both['fut_spread'] = df_both.bk_fut_pr - df_both.fr_fut_pr
df_both['opt_spread'] = df_both.bk_opt_pr - df_both.fr_opt_pr
df_both 

2020-03-13 15:18:25,265 - numexpr.utils - INFO - NumExpr defaulting to 4 threads.


,settle_date,fr_symbol,strike,pc,fr_opt_pr,fr_fut_pr,bk_symbol,bk_opt_pr,bk_fut_pr,fut_spread,opt_spread
0,20180924,ESH19,2750.0,P,50.50,2934.00,ESU19,99.75,2951.50,17.50,49.25
1,20180925,ESH19,2750.0,P,51.50,2930.00,ESU19,100.25,2950.00,20.00,48.75
2,20180926,ESH19,2750.0,P,53.50,2920.25,ESU19,103.00,2940.00,19.75,49.50
3,20180927,ESH19,2750.0,P,51.00,2928.50,ESU19,100.00,2948.50,20.00,49.00
4,20180928,ESH19,2750.0,P,50.75,2927.50,ESU19,100.25,2948.50,21.00,49.50
...,...,...,...,...,...,...,...,...,...,...,...
114,20190311,ESH19,2750.0,P,4.20,2784.00,ESU19,99.00,2793.75,9.75,94.80
115,20190312,ESH19,2750.0,P,1.90,2792.00,ESU19,94.75,2801.50,9.50,92.85
116,20190313,ESH19,2750.0,P,0.40,2814.50,ESU19,86.00,2824.50,10.00,85.60
117,20190314,ESH19,2750.0,P,0.30,2807.25,ESU19,87.75,2817.00,9.75,87.45


In [11]:
graph_cols = ['settle_date','fr_fut_pr','fut_spread','opt_spread']
iplot(plotly_plot(df_in=df_both[graph_cols],x_column='settle_date',
                 yaxis2_cols=['fut_spread','opt_spread']))




### Methods to build implied volatilites

In [12]:
def _get_contract_number_from_symbol(symbol):
    c = symbol[0:2]
    if c in ['CL','CB','ES','GE','NG']:
        return 2
    return 2

In [266]:
USE_PYVOL = True
def lam_pyvol(r):
    try:
        return implied_volatility.implied_volatility(r.close_x,r.close_y,r.strike,.02,r.dte/365, r.pc.lower())
    except:
        return -1
# lam_pyvol = lambda r:implied_volatility.implied_volatility(r.close_x,r.close_y,r.strike,.02,r.dte/365, r.pc.lower())
lam_mibian = lambda r:mibian.BS([r.close_y,r.strike,2,r.dte], callPrice=r.close_x).impliedVolatility

def get_implieds(df,df_expiry_dates,contract,contract_num=2):
    df2 = df[['symbol','contract_num','pc','settle_date','strike','close_x','close_y']]
    df2 = df2[(((df2.pc=='C' )& (df2.strike>=df2.close_y)) | ((df2.pc=='P' ) & (df2.strike<df2.close_y)))  & (df2.symbol.str.contains(contract))]
    cnum = _get_contract_number_from_symbol(contract)
    if contract_num is not None:
        df2 = df2[df2.contract_num==contract_num]
    if len(df2)<1:
        raise ValueError(f"get_implieds: no contracts = contract_num {contract_num}")
    phigh = df2.close_y.max()
    plow = df2.close_y.min()
    high_strike = round(phigh * 1.3)
    low_strike = round(plow * .7)
    df2 = df2[(df2.strike>=low_strike) & (df2.strike<=high_strike)]

    df9 = df2[df2.symbol==contract]
    df9 = df9.merge(df_expiry_dates.rename(columns={'settle_date':'expiry'}),on='symbol',how='inner')
    df9['syear'] = df9.settle_date.astype(str).str.slice(0,4).astype(int)
    df9['smon'] = df9.settle_date.astype(str).str.slice(4,6).astype(int)
    df9['sday'] = df9.settle_date.astype(str).str.slice(6,8).astype(int)
    df9['eyear'] = df9.expiry.astype(str).str.slice(0,4).astype(int)
    df9['emon'] = df9.expiry.astype(str).str.slice(4,6).astype(int)
    df9['eday'] = df9.expiry.astype(str).str.slice(6,8).astype(int)
    df9['sdatetime'] = df9.apply(lambda r:datetime.datetime(r.syear,r.smon,r.sday),axis=1)
    df9['edatetime'] = df9.apply(lambda r:datetime.datetime(r.eyear,r.emon,r.eday),axis=1)
    df9['dte'] = df9.edatetime - df9.sdatetime
    df9.dte = df9.dte.dt.days
    df9 = df9[['symbol','settle_date','pc','contract_num','strike','close_x','close_y','dte']]
    df10 = df9.iloc[:len(df9)].copy()
    df10.index = list(range(len(df10)))
    if USE_PYVOL:
        df10['iv'] = df10.apply(lam_pyvol,axis=1)
    else:
        n = 100
        for i in tqdm_notebook(np.arange(0,len(df10)-n,n)):
                df10.loc[i:i+n,'iv'] = df10.loc[i:i+n].apply(lam_mibian,axis=1)
        print(f'doing remaining {datetime.datetime.now()}')
        i = df10[df10.iv.isna()].index[0]
        df10.loc[i:,'iv'] = df10.loc[i:].apply(lam_mbian,axis=1)
        print(f'done with remaining {datetime.datetime.now()}')
    return df10



#### example of using mibian for options calcs (we use py_vollib instead)

In [14]:
def _test_mibian():
    underlying=1.4565
    strike=1.45
    interest = 1
    days=30
    opt_info = [underlying,strike,interest,days]
    c = mibian.BS(opt_info, volatility=20)
    print(c.callPrice,
    c.putPrice,
    c.callDelta,
    c.putDelta,
    c.callDelta2,
    c.putDelta2,
    c.callTheta,
    c.putTheta,
    c.callRho,
    c.putRho,
    c.vega,
    c.gamma)


    co = mibian.BS(opt_info, callPrice=c.callPrice)
    co.impliedVolatility

In [15]:
_test_mibian()

0.03721154839277063 0.029520257209636247 0.5481584196590104 -0.4518415803409897 -0.52495254471764 0.4742258751560606 -0.0005720853891507385 -0.0005323919998680845 0.000625628375211434 -0.0005651733032681817 0.0016536933299310715 4.742153534821618


#### Show simple example of using py_vol package

In [16]:
def _test_py_vollib():
    #CL,Q2019,560P,07/02/2019,0.6,1.61,0.54,1.54,1997,4465
    F = 56.25
    K = 56
    sigma = .366591539
    flag = 'p'
    t = 15/365.0
    r = .025
    discounted_call_price = black.black(flag, F, K, t, r, sigma)
    dcp = 1.54
    ivpy = implied_volatility.implied_volatility(dcp, F, K, r, t, flag)
    ivmn = mibian.BS([F,K,2.5,15], callPrice=dcp).impliedVolatility
    return discounted_call_price,ivpy,ivmn


In [17]:
_test_py_vollib()

(1.5399999992892466, 0.36659153915713766, 30.517578125)

### Method to get closest-to-the-money and at-the-money (atm) vols

In [18]:
def get_closest_to_the_money_vols(df_implied_vols,
                 opt_close_col='close_x',
                 fut_close_col = 'close_y',
                 iv_col = 'iv'):
    df_in = df_implied_vols.copy()
    df_in['amt_away_from_money'] = df_in.apply(lambda r: abs(r[fut_close_col]-r.strike),axis=1)
    gb_cols = ['settle_date','symbol']
    df_min_amt_away = df_in[gb_cols + ['amt_away_from_money']].groupby(gb_cols,as_index=False).min()
    df_ret = df_min_amt_away.merge(
        df_in,on=gb_cols + ['amt_away_from_money'],how='inner')
    df_lowest_iv = df_ret[['settle_date','symbol','iv']].groupby(
        ['settle_date','symbol'],
        as_index=False).min()
    df_ret = df_ret.merge(df_lowest_iv,on=['settle_date','symbol','iv'],how='inner')
    return df_ret


In [239]:
import pdb

def get_even_moneyness_strikes(df_implieds):
    df10 = df_implieds.copy()
    # define amounts around the money which will help create strikes to add
    moneyness = np.arange(.7,1.4,.05).round(6)
    # define columns on which to execute groupby
#     gb_cols = ['symbol','settle_date','pc','contract_num','dte','close_y']
    gb_cols = ['symbol','settle_date','contract_num','dte','close_y']
    # define function used in groupby.apply to create strikes and iv's at those strikes
    #   where the strikes are an even amount from the money 
    #   (like .7, .8, ... 1, 1.1, 1.2, etc)
    def _add_even_moneyness_strikes(df):
        # get underlying from first row (the groupby makes them all the same)
        r = df.iloc[0]
        underlying = r.close_y
        # create new rows to append to df, using only the gb_cols
        df_ret1 = df.iloc[:len(moneyness)][gb_cols].copy()
        # add nan iv's !!!! MUST BE np.nan - NOT None
        df_ret1['iv'] = np.nan
        # add new strikes
        df_ret1['strike'] = moneyness * underlying
        # append the new strikes
        try:
            # try with using the sort=True options for versions of pandas after 0.23
            dfa = df.append(df_ret1,ignore_index=True,sort=True).copy()
        except:
            # otherwise do not specify sort
            dfa = df.append(df_ret1,ignore_index=True).copy()
        df_ret2 = dfa.sort_values(['symbol','settle_date','pc','strike'])
        df_ret2 = df_ret2.drop_duplicates(subset='strike')
        # set the index to the strike so that interpolate works
        df_ret2.index = df_ret2.strike
        # create interpolated iv's
        df_ret2['iv'] = df_ret2.iv.interpolate(method='polynomial', order=2)
        # reset the index
        df_ret2.index = list(range(len(df_ret2)))
        return df_ret2

    # start here
    df11 = df10.groupby(gb_cols).apply(_add_even_moneyness_strikes).copy()
    df11.index = list(range(len(df11)))
    df11['moneyness'] = df11.strike / df11.close_y
    df11.moneyness = df11.moneyness.round(4)

    df12 = df11[(df11.moneyness.isin(moneyness)) & (~df11.iv.isna())].copy()
    df12.moneyness  = df12.moneyness - 1
    df12.index = list(range(len(df12)))
    df12_atm = df12[df12.moneyness==0][['symbol','settle_date','pc','iv']]
    df12_atm = df12_atm.rename(columns={'iv':'atm_iv'})
    
    df12_atm = df12_atm.drop_duplicates()
    df12_atm.pc = ''#np.nan
    df12.pc = ''#np.nan

    df12 = df12.merge(df12_atm,on=['symbol','settle_date','pc'],how='inner')

#     df12 = df12.merge(df12_atm,on=['symbol','settle_date'],how='inner')
#     df12['pc'] = np.nan

    df12.moneyness = df12.moneyness.round(4)
    df12['vol_skew'] = (df12.iv - df12.atm_iv).round(4)
    # now get rid of cases where an even money strike, and an actual strike are the same, so that you have 
    #    2 vol_skew records for the same 'settle_date','symbol','moneyness' combination
    # THIS IS THE EASY WAY
#     df12 = df12[df12.close_x.isna()]
    # THIS IS THE HARD WAY
    gb_cols = ['settle_date','symbol','moneyness','close_y','contract_num','dte']
    val_cols = ['vol_skew','iv','strike','atm_iv']
    df12 = df12[gb_cols + val_cols].groupby(gb_cols,as_index=False).mean()

    return df12



### Method to get volatility skews per symbol

In [267]:
def create_skew_per_date_df(df):
    '''
    Find the first settle_date whose count of rows is equal to max count of rows.
    '''
    # get the first symbol (which should be the only symbol)
    contract = df.symbol.unique()[0]
    # get just that symbol's data
    df12 = df[df.symbol==contract]
    df_counts = df12[['settle_date','moneyness']].groupby('settle_date',as_index=False).count()
    max_count = df_counts.moneyness.max()
    first_max_count_settle_date = df_counts[df_counts.moneyness==max_count].iloc[0].settle_date
    
    df_ret = df12[df12.settle_date==first_max_count_settle_date][['moneyness']]
    all_settle_dates = sorted(df_counts.settle_date.unique())
    for settle_date in all_settle_dates:
        df_temp = df12[df12.settle_date==settle_date][['moneyness','vol_skew']]
        df_ret = df_ret.merge(df_temp,on='moneyness',how='outer')
        df_ret = df_ret.rename(columns={'vol_skew':str(settle_date)})
    df_ret = df_ret.sort_values('moneyness')
    df_ret.moneyness = df_ret.moneyness.round(4)
    return df_ret


def skew_per_symbol(symbol,strike_divisor=None,contract_num=2):
    '''
    For a symbol like CLM16 or EZH19, create 2 Dataframes
      1. df_iv - contains rows of implied vols, for only the 'pseudo' strikes that are an even
                 percent away from the money for each settle_date
      2. df_skew - contains one row per day of skew data of for 'pseudo' strikes that are an even
                 percent away from the money for each settle_date
    '''
    _exception = None
    _stacktrace = None
    df_iv = None
    df_skew = None
    df,df_expiry_dates = get_postgres_data(symbol)
    if len(df[df.contract_num==contract_num])>0:
        df10 = get_implieds(df,df_expiry_dates,symbol,contract_num)
        df12 = get_even_moneyness_strikes(df10)
        df_sk = create_skew_per_date_df(df12)
        df_sk.index = list(range(len(df_sk)))
        df_skt = df_sk.T
        df_skt.columns = df_skt.loc['moneyness']
        df_skt = df_skt.iloc[1:].copy()
        df_skt['symbol'] = symbol
        df_skt['settle_date'] = df_skt.index
        df_iv = df12.copy() 
        df_skew = df_skt.copy()
        df_skew = df_skew.sort_values('settle_date')
        df_skew.index = list(range(len(df22)))

    return df_iv,df_skew

### test getting implieds for a single contract

In [269]:
strike_div = None if SYMBOL_TO_RESEARCH not in STRIKE_DIVISORS.keys() else STRIKE_DIVISORS[SYMBOL_TO_RESEARCH]
sym = "ESZ20"
contract_num = 2 if sym != 'ESZ20' else 4
df,df_expiry_dates = get_postgres_data(sym)
df_implieds = get_implieds(df,df_expiry_dates,sym,contract_num=contract_num)
df_vols = None
df_ctm_vols = None
df_atm_money = None
if df_implieds is not None:
    df_ctm_vols = get_closest_to_the_money_vols(df_implieds)
    df_even_money = get_even_moneyness_strikes(df_implieds)
    df_atm_money = df_even_money[df_even_money.moneyness==0]
    df_vols = df_ctm_vols.merge(
        df_atm_money[['settle_date','symbol','atm_iv']],
        on=['settle_date','symbol'],how='inner')
else:
    print(f'no implied vols for symbol: {sym}')

In [270]:
df

,symbol,strike,pc,settle_date,open_x,high_x,low_x,close_x,adj_close_x,volume_x,open_interest_x,u_symbol,contract_num,open_y,high_y,low_y,close_y,adj_close_y,volume_y,open_interest_y
0,ESZ20,1600.0,C,20191223,0.0,1626.00,1625.90,1626.00,1626.00,0,0,ESZ20,4,3229.5,3229.50,3226.0,3226.00,3226.00,2,2226
1,ESZ20,1600.0,P,20191223,0.0,3.20,3.20,3.20,3.20,0,0,ESZ20,4,3229.5,3229.50,3226.0,3226.00,3226.00,2,2226
2,ESZ20,1700.0,C,20191223,0.0,1526.00,1525.90,1526.00,1526.00,0,0,ESZ20,4,3229.5,3229.50,3226.0,3226.00,3226.00,2,2226
3,ESZ20,1700.0,P,20191223,0.0,4.40,4.40,4.40,4.40,0,0,ESZ20,4,3229.5,3229.50,3226.0,3226.00,3226.00,2,2226
4,ESZ20,1800.0,C,20191223,0.0,1426.10,1426.00,1426.00,1426.00,0,0,ESZ20,4,3229.5,3229.50,3226.0,3226.00,3226.00,2,2226
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6991,ESZ20,4200.0,P,20200313,0.0,1533.30,1533.25,1533.25,1533.25,0,0,ESZ20,4,2400.0,2668.25,2383.0,2668.25,2668.25,18,2580
6992,ESZ20,4300.0,C,20200313,0.0,1.95,1.95,1.95,1.95,0,1783,ESZ20,4,2400.0,2668.25,2383.0,2668.25,2668.25,18,2580
6993,ESZ20,4300.0,P,20200313,0.0,1632.80,1632.75,1632.75,1632.75,0,0,ESZ20,4,2400.0,2668.25,2383.0,2668.25,2668.25,18,2580
6994,ESZ20,4400.0,C,20200313,0.9,1.60,0.90,1.60,1.60,0,426,ESZ20,4,2400.0,2668.25,2383.0,2668.25,2668.25,18,2580


In [271]:
df_even_money

,settle_date,symbol,moneyness,close_y,contract_num,dte,vol_skew,iv,strike,atm_iv
0,20191223,ESZ20,-0.30,3226.00,4,361,0.0971,0.252048,2258.2000,0.154955
1,20191223,ESZ20,-0.25,3226.00,4,361,0.0833,0.238234,2419.5000,0.154955
2,20191223,ESZ20,-0.20,3226.00,4,361,0.0691,0.224012,2580.8000,0.154955
3,20191223,ESZ20,-0.15,3226.00,4,361,0.0540,0.208982,2742.1000,0.154955
4,20191223,ESZ20,-0.10,3226.00,4,361,0.0381,0.193006,2903.4000,0.154955
...,...,...,...,...,...,...,...,...,...,...
714,20200313,ESZ20,0.15,2668.25,4,280,-0.0902,0.235787,3068.4875,0.325957
715,20200313,ESZ20,0.20,2668.25,4,280,-0.1196,0.206348,3201.9000,0.325957
716,20200313,ESZ20,0.25,2668.25,4,280,-0.1417,0.184243,3335.3125,0.325957
717,20200313,ESZ20,0.30,2668.25,4,280,-0.1475,0.178478,3468.7250,0.325957


In [22]:
sql_all_contracts = f"""
select distinct symbol from {opttab} 
where substring(symbol,1,2) = 'ES' and
substring(symbol,3,1) in ('H','M','U','Z')
"""
all_contracts = sorted(pga.get_sql(sql_all_contracts).symbol.values)


In [23]:
all_contracts_reversed = sorted([f'{c[:2]}{c[-2:]}{c[2]}' for c in all_contracts])
all_contracts2 = [f'{c[:2]}{c[-1]}{c[2:4]}' for c in all_contracts_reversed]
all_contracts2

['ESH11',
 'ESM11',
 'ESU11',
 'ESZ11',
 'ESH12',
 'ESM12',
 'ESU12',
 'ESZ12',
 'ESH13',
 'ESM13',
 'ESU13',
 'ESZ13',
 'ESH14',
 'ESM14',
 'ESU14',
 'ESZ14',
 'ESH15',
 'ESM15',
 'ESU15',
 'ESZ15',
 'ESH16',
 'ESM16',
 'ESU16',
 'ESZ16',
 'ESH17',
 'ESM17',
 'ESU17',
 'ESZ17',
 'ESH18',
 'ESM18',
 'ESU18',
 'ESZ18',
 'ESH19',
 'ESM19',
 'ESU19',
 'ESZ19',
 'ESH20',
 'ESM20',
 'ESU20',
 'ESZ20']

In [24]:
sbstrike = .1325
sb = .1172
sbcall = .0006
sbput = sbcall + sbstrike - sb
sbput, sbstrike - sb,sbput + sb

(0.015899999999999997, 0.015300000000000008, 0.1331)

['ESH17', 'ESM17', 'ESU17', 'ESZ17']

In [283]:
begin_year = 20

In [286]:
def get_all_implied_info(symbol,contract_num = 2):
    df_implieds = None
    df_even_money = None
    df_vols = None
    ret = {'implieds':df_implieds,'even_money':df_even_money,'vols':df_vols}
    commod = symbol[0:2]
    strike_div = None if commod not in STRIKE_DIVISORS.keys() else STRIKE_DIVISORS[commod]
    df,df_expiry_dates = get_postgres_data(symbol)
    if df is None or len(df)<10:
        print(f'no data for symbol: {symbol}')
        return {'implieds':None,'even_money':None,'vols':None}
    df_implieds = get_implieds(df,df_expiry_dates,symbol,contract_num=contract_num)
    if df_implieds is not None:
        df_ctm_vols = get_closest_to_the_money_vols(df_implieds)
        df_even_money = get_even_moneyness_strikes(df_implieds)
        df_atm_money = df_even_money[df_even_money.moneyness==0]
        df_vols = df_ctm_vols.merge(
            df_atm_money[['settle_date','symbol','atm_iv']],
            on=['settle_date','symbol'],how='inner')
    return {'hist_data':df,'implieds':df_implieds,'even_money':df_even_money,'vols':df_vols}

for sym in [c for c in all_contracts2 if int(c[-2:])>=begin_year]:
    print(f"graphing symbol: {sym}")
    commod = sym[0:2]
    strike_div = None if commod not in STRIKE_DIVISORS.keys() else STRIKE_DIVISORS[commod]
#     df,df_expiry_dates = get_postgres_data(sym)
#     if df is None or len(df)<10:
#         print(f'no data for symbol: {sym}')
#         continue
    
    try:
        d = get_all_implied_info(sym,contract_num=None)
    except Exception as e:
        print(f'Exception getting implied vols for symbol: {sym}')
        print(str(e))
        continue
    df_implieds = d['implieds']
    if df_implieds is None:
        print(f'no implied vols for symbol: {sym}')
        continue
    df_vols = d['vols']
    
    iplot(plotly_plot(
        df_in=df_vols[['settle_date','close_y','atm_iv']],
        x_column='settle_date',yaxis2_cols=['atm_iv'],number_of_ticks_display=15)
         )    

graphing symbol: ESH20


graphing symbol: ESM20


graphing symbol: ESU20


graphing symbol: ESZ20


### Options strategy history

In [287]:
def ot_from_sn(option_short_name):
    '''
    get OptTrade instance from option_short_name string like 'ESM20_2500_C_-4'
    YOU must have at least 'ESM20_2500_C'.  If no quantity, it will be zero.
    '''
    if option_short_name is None:
        return None
    parts = option_short_name.split('_')
    if len(parts)<3:
        return None
    sym = parts[0]
    strike = parts[1]
    pc = parts[2]
    qty = 0 if len(parts)<4 else float(str(parts[3]))
    return OptTrade(sym,strike,pc,qty)

class OptTrade():
    def __init__(self,symbol,strike,pc,qty,pga=None):
        self.symbol = symbol
        self.strike=strike
        self.pc = pc.upper()
        self. qty = qty
    def __str__(self):
        return f"{self.symbol}_{self.strike}_{self.pc}_{self.qty}"   
    def get_postgres_data(self):
        df,df_expiry = get_postgres_data(self.symbol)
        if df is None or len(df)<1:
            return None
        df = df[df.pc.str.lower()==self.pc.lower()]
        df = df[df.strike.astype(float).round(5)==round(float(str(self.strike)),5)]
        df.index = list(range(len(df)))
        return df
    def get_implied_info(self,contract_num=2):
        return get_all_implied_info(self.symbol,contract_num=contract_num)

legs = ['ESM20_2500_P_-1','ESZ20_2500_P_1']
ot_legs = [ot_from_sn(leg) for leg in legs]
[str(ot_leg) for ot_leg in ot_legs]



['ESM20_2500_P_-1.0', 'ESZ20_2500_P_1.0']

In [280]:
df_leg0_even_money = ot_legs[0].get_implied_info(contract_num=None)['even_money']
df_leg1_even_money = ot_legs[1].get_implied_info(contract_num=None)['even_money']


In [282]:
df_leg0_even_money

,settle_date,symbol,moneyness,close_y,contract_num,dte,vol_skew,iv,strike,atm_iv
0,20190624,ESM20,-0.30,2958.0,4,361,0.1011,0.250625,2070.6,0.149553
1,20190624,ESM20,-0.25,2958.0,4,361,0.0847,0.234281,2218.5,0.149553
2,20190624,ESM20,-0.20,2958.0,4,361,0.0691,0.218637,2366.4,0.149553
3,20190624,ESM20,-0.15,2958.0,4,361,0.0527,0.202282,2514.3,0.149553
4,20190624,ESM20,-0.10,2958.0,4,361,0.0362,0.185734,2662.2,0.149553
...,...,...,...,...,...,...,...,...,...,...
2312,20200313,ESM20,0.15,2684.0,2,98,-0.1616,0.277830,3086.6,0.439418
2313,20200313,ESM20,0.20,2684.0,2,98,-0.1950,0.244388,3220.8,0.439418
2314,20200313,ESM20,0.25,2684.0,2,98,-0.1974,0.242016,3355.0,0.439418
2315,20200313,ESM20,0.30,2684.0,2,98,-0.1877,0.251713,3489.2,0.439418


In [279]:
def _black(r):
    flag = 'p' if r.moneyness < 0 else 'c'
    F = r.close_y
    K = r.strike
    t = r.dte/365
    rate = .015
    sigma = r.atm_iv + r.vol_skew
    return black.black(flag, F, K, t, rate, sigma)
df_leg0_even_money2 = df_leg0_even_money.copy()
df_leg0_even_money2['optprice_leg0'] = df_leg0_even_money2.apply(_black,axis=1)

df_leg1_even_money2 = df_leg1_even_money.copy()
df_leg1_even_money2['optprice_leg1'] = df_leg1_even_money2.apply(_black,axis=1)
both_merge_cols = ['settle_date','moneyness']
df_leg0_even_money2 = df_leg0_even_money2[both_merge_cols + ['optprice_leg0']]
df_leg1_even_money2 = df_leg1_even_money2[both_merge_cols + ['optprice_leg1']]
df_both_legs = df_leg0_even_money2.merge(df_leg1_even_money2,on=both_merge_cols,how='inner')
df_both_legs['optprice_spread'] = df_both_legs.optprice_leg1 - df_both_legs.optprice_leg0
df_both_legs[df_both_legs.moneyness==-.15]

,settle_date,moneyness,optprice_leg0,optprice_leg1,optprice_spread
3,20191223,-0.15,31.886741,75.305024,43.418283
15,20191224,-0.15,31.526492,75.378477,43.851986
27,20191226,-0.15,30.416339,74.398836,43.982497
39,20191227,-0.15,31.518467,75.433538,43.915071
51,20191230,-0.15,32.315385,77.476021,45.160636
63,20191231,-0.15,30.218630,74.865965,44.647335
75,20200102,-0.15,27.821272,71.997851,44.176579
87,20200103,-0.15,29.814738,74.406374,44.591636
99,20200106,-0.15,28.765809,73.606171,44.840361
111,20200107,-0.15,28.550489,72.221009,43.670520


### Create strategy example from ot_legs above

In [73]:
# d1,d2  = get_postgres_data(ot_legs[0].symbol)
# d1
ot_legs[0].get_postgres_data()

,symbol,strike,pc,settle_date,open_x,high_x,low_x,close_x,adj_close_x,volume_x,open_interest_x,u_symbol,contract_num,open_y,high_y,low_y,close_y,adj_close_y,volume_y,open_interest_y
0,ESM20,2500.0,P,20190624,0.00,61.50,61.40,61.50,61.50,0,0,ESM20,4,0.00,2958.00,2958.00,2958.00,2958.00,0,24
1,ESM20,2500.0,P,20190625,62.50,67.75,61.75,67.75,67.75,0,0,ESM20,4,0.00,2924.75,2924.75,2924.75,2924.75,0,169
2,ESM20,2500.0,P,20190626,66.50,68.75,66.50,68.75,68.75,200,200,ESM20,4,0.00,2926.50,2921.00,2921.00,2921.00,0,169
3,ESM20,2500.0,P,20190627,0.00,68.00,65.25,65.25,65.25,501,701,ESM20,4,2921.00,2935.00,2921.00,2935.00,2935.00,0,904
4,ESM20,2500.0,P,20190628,63.75,63.75,62.00,62.00,62.00,501,701,ESM20,4,2938.00,2951.50,2938.00,2947.75,2947.75,6,1143
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
177,ESM20,2500.0,P,20200306,55.50,98.00,53.00,73.00,73.00,8349,0,ESM20,2,3015.25,3028.00,2887.75,2953.75,2953.75,121737,188094
178,ESM20,2500.0,P,20200309,115.00,155.00,115.00,143.00,143.00,3338,19838,ESM20,2,2901.25,2907.25,2698.00,2735.25,2735.25,124994,213911
179,ESM20,2500.0,P,20200310,123.25,150.50,88.00,105.75,105.75,1713,20047,ESM20,2,2721.25,2873.00,2683.25,2854.25,2854.25,160999,263323
180,ESM20,2500.0,P,20200311,128.75,165.00,128.00,149.50,149.50,6441,21236,ESM20,2,2849.75,2858.00,2691.25,2729.00,2729.00,195661,299064


In [357]:
def _black(r):
    flag = 'p' if r.moneyness < 0 else 'c'
    F = r.close_y
    K = r.strike
    t = r.dte/365
    rate = .015
    sigma = r.atm_iv + r.vol_skew
    return black.black(flag, F, K, t, rate, sigma)

class MultiLeg():
    '''
    create a history of a multi-leg option
    '''
    opt_cols = ['strike','pc','open_x','high_x','low_x','close_x',
              'adj_close_x','volume_x','open_interest_x']
    fut_cols = ['open_y','high_y','low_y','close_y',
              'adj_close_y','volume_y','open_interest_y']
    both_cols = ['settle_date','symbol']
    def __init__(self,leg_list: list,contract_num_list=None):
        self.leg_list = leg_list
        self.contract_num_list = [None for _ in range(len(leg_list))] if contract_num_list is None else contract_num_list
        # get history for each leg
        hist_list = []
        for i in range(len(leg_list)):
            leg = leg_list[i]
            contract_num = self.contract_num_list[i]
            print(f"MultiLeg geting data for leg {str(leg)} ")
            hist_dict = leg.get_implied_info(contract_num=contract_num)
            hist_list.append(hist_dict)
        self.hist_list = hist_list

        
        
    def get_values(self,col_list):
        yyyymmdd_set = None

        for i in range(len(self.hist_list)):
            df = self.hist_list[i]['hist_data']
            yyyymmdd_set_df = set(df.settle_date.values)
            if yyyymmdd_set is None:
                yyyymmdd_set = yyyymmdd_set_df
            else:
                if len(yyyymmdd_set)>len(yyyymmdd_set_df):
                    yyyymmdd_set = yyyymmdd_set.intersection(yyyymmdd_set_df)
                else:
                    yyyymmdd_set = yyyymmdd_set_df.intersection(yyyymmdd_set)
            # save all data for this eg
        yyyymmdd_set = list(yyyymmdd_set)
        hist_df_list = []
        for i in range(len(self.leg_list)):
            leg = self.leg_list[i]
            df = self.hist_list[i]['hist_data'].copy()
            df['short_name'] = str(leg)
            df = df[(df.strike==float(str(leg.strike))) & (df.pc.str.upper()==leg.pc.upper())]
            df = df[df.settle_date.isin(yyyymmdd_set)]
            df.index = list(range(len(df)))
            hist_df_list.append(df)

        df = hist_df_list[0][['settle_date']]
        for c in col_list:
            c_values = np.zeros(len(df))
            for i in range(len(self.leg_list)):
                leg = self.leg_list[i]
                qty = float(leg.qty)
                df_hist = hist_df_list[i].copy()
                df_hist[c].fillna(0)
                c_values = c_values + df_hist[c].values * qty
            df[c] = c_values
        return df
    
    def get_theo_values(self):
        df_all_legs = None
        both_merge_cols = ['settle_date','moneyness']
        # get the even_money dataframe for each leg, and get theo prices from it
        for i in range(len(self.hist_list)):
            # get the dictionary of returned history values
            hist_dict = self.hist_list[i]
            # get the dataframe with the even money strike values
            df_even_money = hist_dict['even_money'].copy()
            # create a list of price cols for each leg
            optprice_col = f"optprice_leg_{i}"
            # get a black scholes option price for each even money strike
            df_even_money[optprice_col] = df_even_money.apply(_black,axis=1)
            # limit the columns to the merge cols and the optprice col for this leg
            df_even_money = df_even_money[both_merge_cols +[optprice_col]]
            # merge all the leg option values into one dataframe
            if df_all_legs is None:
                df_all_legs = df_even_money.copy()
            else:
                df_all_legs = df_all_legs.merge(df_even_money,on=both_merge_cols,how='inner')

        # now combine the optprices
        op_cols = [f"optprice_leg_{i}" for i in range(len(self.hist_list))]
        df_all_legs['strat_price'] = 0
        for i in range(len(op_cols)):
            c = op_cols[i]
            leg = self.leg_list[i]
            qty = leg.qty
            df_all_legs.strat_price = df_all_legs.strat_price + df_all_legs[c] * qty

        return df_all_legs
                    
                    
        
# df_leg0_even_money2 = df_leg0_even_money.copy()
# df_leg0_even_money2['optprice_leg0'] = df_leg0_even_money2.apply(_black,axis=1)

# df_leg1_even_money2 = df_leg1_even_money.copy()
# df_leg1_even_money2['optprice_leg1'] = df_leg1_even_money2.apply(_black,axis=1)
# both_merge_cols = ['settle_date','moneyness']
# df_leg0_even_money2 = df_leg0_even_money2[both_merge_cols + ['optprice_leg0']]
# df_leg1_even_money2 = df_leg1_even_money2[both_merge_cols + ['optprice_leg1']]
# df_both_legs = df_leg0_even_money2.merge(df_leg1_even_money2,on=both_merge_cols,how='inner')
# df_both_legs['optprice_spread'] = df_both_legs.optprice_leg1 - df_both_legs.optprice_leg0
# df_both_legs[df_both_legs.moneyness==-.15]



In [358]:
es_m0z0_2500 = MultiLeg(ot_legs)

MultiLeg geting data for leg ESM20_2500_P_-1.0 
MultiLeg geting data for leg ESZ20_2500_P_1.0 


In [359]:
es_m0z0_2500.get_values(['close_x','open_x','high_x','low_x'])

,settle_date,close_x,open_x,high_x,low_x
0,20191223,28.00,28.00,28.25,28.00
1,20191224,28.00,-15.00,28.25,28.00
2,20191226,27.25,26.50,26.50,27.25
3,20191227,28.00,26.50,28.00,27.00
4,20191230,29.25,30.00,29.00,30.00
5,20191231,28.50,-15.00,28.75,28.50
6,20200102,27.00,27.75,27.25,26.75
7,20200103,28.50,32.25,30.75,27.75
8,20200106,27.75,-14.50,30.25,27.50
9,20200107,27.00,26.75,27.25,27.50


In [362]:
df_es_m0z0_2500_theo_values  = es_m0z0_2500.get_theo_values()
df_es_m0z0_2500_theo_values[df_es_m0z0_2500_theo_values.moneyness==-.15]

,settle_date,moneyness,optprice_leg_0,optprice_leg_1,strat_price
3,20191223,-0.15,31.886741,75.305024,43.418283
15,20191224,-0.15,31.526492,75.378477,43.851986
27,20191226,-0.15,30.416339,74.398836,43.982497
39,20191227,-0.15,31.518467,75.433538,43.915071
51,20191230,-0.15,32.315385,77.476021,45.160636
63,20191231,-0.15,30.218630,74.865965,44.647335
75,20200102,-0.15,27.821272,71.997851,44.176579
87,20200103,-0.15,29.814738,74.406374,44.591636
99,20200106,-0.15,28.765809,73.606171,44.840361
111,20200107,-0.15,28.550489,72.221009,43.670520


In [190]:
es_m19_2000_leg = ot_from_sn('ESM19_2000_P_-1')
es_z19_2000_leg = ot_from_sn('ESZ19_2000_P_1')
es_m9z9_2000 = MultiLeg([es_m19_2000_leg,es_z19_2000_leg])


In [191]:
df_es_m9z9_2000 = es_m9z9_2000.get_values(['close_x'])
df_m9 = es_m9z9_2000.hist_list[0][['settle_date','close_y']]
df_z9 = es_m9z9_2000.hist_list[1][['settle_date','close_y']]
df_es_m9z9_2000 = df_es_m9z9_2000.merge(df_m9,on='settle_date',how='inner')
df_es_m9z9_2000 = df_es_m9z9_2000.merge(df_z9,on='settle_date',how='inner')
df_es_m9z9_2000



,settle_date,close_x,close_y_x,close_y_y
0,20181224,32.75,2344.25,2352.75
1,20181226,27.75,2474.25,2484.50
2,20181227,27.50,2498.25,2506.25
3,20181228,27.75,2489.25,2498.75
4,20181231,27.50,2508.50,2514.50
...,...,...,...,...
119,20190617,5.45,2892.00,2897.50
120,20190618,5.20,2921.75,2927.75
121,20190619,4.95,2929.00,2934.25
122,20190620,4.85,2955.75,2960.50


### get skew per leg

In [364]:
10/2990

0.0033444816053511705

## END